In [ ]:
print('Hello')

In [ ]:

# # # Install Open Cv - PIP - tool to install packages meaning PIP Install Packages/Python pip install opencv-python
# pip install opencv-python
# pip install deepface
# pip install fer -- version 22.4.0 and moviepy 1.0.3


In [ ]:
# # checking if its installed correctly and understanding the version
import cv2
print(cv2.__version__)
import os

from deepface import DeepFace
import moviepy
print("moviepy is working!")

# from fer import FER
# print(fer.__version__)


# import tensorflow as tf
# print(deepface.__version__)

In [ ]:
video = cv2.VideoCapture('/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/smile.mp4')

if not video.isOpened():
    print("❌ Error opening video file.")
else:
    print("✅ Video successfully loaded!");
    ret, frame = video.read()
    print('ret', ret, 'frame' ,frame, video)
    if ret:
        print('✅ Frame extracted!');
        cv2.imwrite('first_frame.jpg',frame)
    else:
        print("❌ Error reading video file.")
    
video.release();

In [ ]:
# to extract all the frames in a folder 
# Always open video again - dont skip this step 
video = cv2.VideoCapture('/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/smile.mp4')

if not video.isOpened():
    print("❌ Error opening video file.")
smile_frames ="smile frames"
if not os.path.exists(smile_frames):
    os.makedirs(smile_frames)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

for frame_num in range (1, frame_count+1):
    ret, frame = video.read()
    if not ret:
        print("❌ Error reading video file.")
        print(f"Stopped at frame {frame_num}")
        break
    frame_filename = os.path.join(smile_frames, f"frame_{frame_num}.jpg")
    cv2.imwrite(frame_filename,frame)
    print(f"Saved {frame_filename}")

# video.release()
# print("✅ All frames extracted and saved using a for loop.")
        

In [ ]:
frames_folder = "smile frames"
frames_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg')] )

for frame_file in frames_files:
    frame_path = os.path.join(frames_folder, frame_file)
    frame = cv2.imread(frame_path)

    try:
        result = DeepFace.analyze(frame, actions =['emotion'], enforce_detection = False)
        emotion = result[0]['dominant_emotion'] if isinstance(result,list) else result['dominant_emotion']
        print(f"{frame_file}: {emotion}")
    except Exception as e:
        print(f"Could not process {frame_file}: {e}")
        

# There is an ongoing issue with the package - not processing images well

In [ ]:
# I am creating a new pipeline using different packages to recognise faces and emotions

In [ ]:
# creating frames for the video 
import cv2
import os

# Path to your video
video_path = "/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4"
frames_folder = "video_frames"

# Create folder for frames if it doesn't exist
if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)
print(f"Frames will be saved in: {frames_folder}")
# Open the video file
cap = cv2.VideoCapture(video_path)

# Read and save frames
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = os.path.join(frames_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()

In [ ]:
import cv2
print(cv2.data.haarcascades)

In [ ]:
# didnt work as expected - the accuracy with which it captured the faces is too low. 
import cv2
import os

frames_folder = "video_frames"
faces_folder = "cropped_faces"

if not os.path.exists(faces_folder):
    os.makedirs(faces_folder)

# Load the face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

for frame_name in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_name)
    img = cv2.imread(frame_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # For each detected face, crop and save
    for i, (x, y, w, h) in enumerate(faces):
        face_img = img[y:y+h, x:x+w]
        out_file = os.path.join(faces_folder, f"{frame_name[:-4]}_face_{i+1}.jpg")
        cv2.imwrite(out_file, face_img)
        print(f"Saved {out_file}")

print("✅ All faces cropped and saved.")

In [ ]:
from mtcnn import MTCNN
import cv2
import os

frames_folder = "video_frames"
faces_folder = "cropped_faces_mtcnn"

if not os.path.exists(faces_folder):
    os.makedirs(faces_folder)

detector = MTCNN()

for frame_name in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_name)
    img = cv2.imread(frame_path)
    results = detector.detect_faces(img)
    for i, res in enumerate(results):
        x, y, w, h = res['box']
        face_img = img[y:y+h, x:x+w]
        out_file = os.path.join(faces_folder, f"{frame_name[:-4]}_face_{i+1}.jpg")
        cv2.imwrite(out_file, face_img)
        print(f"Saved {out_file}")

In [ ]:
from deepface import DeepFace
import os

faces_folder = "cropped_faces_mtcnn"  # Or your folder name
embeddings = {}  # filename -> embedding

for img_name in os.listdir(faces_folder):
    img_path = os.path.join(faces_folder, img_name)
    try:
        # Get embedding vector (using VGG-Face model by default)
        embedding = DeepFace.represent(img_path=img_path, model_name='VGG-Face', enforce_detection=False)[0]["embedding"]
        embeddings[img_name] = embedding
    except Exception as e:
        print(f"Error for {img_name}: {e}")

print("✅ All embeddings computed.")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def is_same_person(emb1, emb2, threshold=0.7):
    similarity = cosine_similarity([emb1], [emb2])[0][0]
    return similarity > threshold

In [ ]:
# Let's assign a "person ID" to each unique face
from collections import defaultdict


if 'embeddings' not in globals():
    raise NameError("Variable 'embeddings' is not defined. Please run the cell that defines 'embeddings' first.")

person_id = 1
face_to_person = {}
person_to_embeddings = defaultdict(list)

for img1, emb1 in embeddings.items():
    assigned = False
    for pid, emb_list in person_to_embeddings.items():
        # Check if similar to ANY embedding already in that cluster/person
        if any(is_same_person(emb1, e) for e in emb_list):
            face_to_person[img1] = pid
            person_to_embeddings[pid].append(emb1)
            assigned = True
            break
    if not assigned:
        face_to_person[img1] = person_id
        person_to_embeddings[person_id].append(emb1)
        person_id += 1

print(face_to_person)

# Now face_to_person contains mapping of face images to unique person IDs!

In [ ]:
def is_same_person(emb1, emb2, threshold=0.5):  # Try 0.5 or even 0.45
    from sklearn.metrics.pairwise import cosine_similarity
    return cosine_similarity([emb1], [emb2])[0,0] > threshold

In [ ]:
from deepface import DeepFace

emotions_result = {}

for img_file in embeddings.keys():
    img_path = f"cropped_faces_mtcnn/{img_file}"
    try:
        result = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)
        # Handle if result is a list (common with DeepFace >=0.0.78)
        if isinstance(result, list):
            result = result[0]
        emotions_result[img_file] = result['emotion']
    except Exception as e:
        print(f"Error processing {img_file}: {e}")
        emotions_result[img_file] = None

In [ ]:
person_emotions = {}

for img_file, person_id in face_to_person.items():
    emotion = emotions_result.get(img_file)
    if emotion is not None:
        if person_id not in person_emotions:
            person_emotions[person_id] = []
        person_emotions[person_id].append({'img_file': img_file, 'emotion': emotion})

In [ ]:
for item in person_emotions .items():
    person_id, emotions = item
    print(f"Person ID: {person_id}")
    for emotion in emotions:
        print(f"  Image: {emotion['img_file']}, Emotion: {emotion['emotion']}")


In [ ]:
from collections import defaultdict, Counter

# Step 1: Group all top emotions by person
person_emotions = defaultdict(list)

for img_file, person_id in face_to_person.items():
    emotion_dict = emotions_result.get(img_file)
    if emotion_dict:
        top_emotion = max(emotion_dict, key=emotion_dict.get)
        person_emotions[person_id].append(top_emotion)

In [ ]:
for person_id, emotions in person_emotions.items():
    if emotions:
        # Counter counts how many times each emotion appears
        most_common = Counter(emotions).most_common(1)[0]
        print(f"Person {person_id}: Predominant emotion is {most_common[0]} (appeared {most_common[1]} times)")
    else:
        print(f"Person {person_id}: No emotions detected.")

In [ ]:
# Step-by-step Workflow for Bounding & Tracking

# 1. Detect Faces and Get Bounding Boxes
# 	•	Use a face detector (like MTCNN, RetinaFace, or OpenCV DNN) to detect faces and get bounding boxes for every frame.

# 2. Track Faces Across Frames
# 	•	Use a tracker (like SORT, DeepSORT, or even simple IOU matching) to assign a unique ID to each bounding box, so that the same person keeps the same ID as they move frame-to-frame.

# 3. Analyze Emotion Within Each Bounding Box
# 	•	For each bounding box (i.e., each person in each frame), crop that region (just in memory, you don’t need to save to disk) and pass it to your emotion detector (DeepFace, FER, etc).

# 4. Aggregate Emotions Per Person (ID)
# 	•	For each unique ID, collect emotions across frames for stats and visualization.

# ⸻


In [ ]:
import cv2
import os
from mtcnn import MTCNN

# Load the video
video_path = "/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4"

output_folder = "frames_with_boxes"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

cap = cv2.VideoCapture(video_path)
detector = MTCNN()
frame_num = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = detector.detect_faces(frame)
    
    for result in results:
        x, y, width, height = result['box']
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
    
    # Save frame with bounding boxes
    frame_filename = os.path.join(output_folder, f"frame_{frame_num:04d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_num += 1

cap.release()
cv2.destroyAllWindows()
print(f"✅ All frames with bounding boxes saved in folder: '{output_folder}'")

In [ ]:
# didnt seem to work as expected - the accuracy with which it captured the faces is too low.# I am creating a new pipeline using different packages to recognise faces and emotions
import cv2
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


model = YOLO('/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/emotion_aware_coach_assistant_practice/yolov8n.pt', task='detect')  # Load the YOLO model for face detection
tracker = DeepSort(max_age=30)
# Create output directory for frames
output_dir = "output_frames"
os.makedirs(output_dir, exist_ok=True)
cap = cv2.VideoCapture("/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4")
frame_num = 0
pad = 20  # pixels of padding around face

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = []
    for r in results:
        for box, conf, cls in zip(r.boxes.xyxy, r.boxes.conf, r.boxes.cls):
            # All detections are faces, so no filtering needed!
            x1 = max(int(box[0]) - pad, 0)
            y1 = max(int(box[1]) - pad, 0)
            x2 = min(int(box[2]) + pad, frame.shape[1])
            y2 = min(int(box[3]) + pad, frame.shape[0])
            detections.append(([x1, y1, x2 - x1, y2 - y1], float(conf), 'face'))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

    cv2.imshow("Tracked Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frame_num += 1


# Only save the last frame if it was successfully read
if ret and frame is not None:
    cv2.imwrite(os.path.join(output_dir, f"frame_{frame_num:04d}.jpg"), frame)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from retinaface import RetinaFace
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# 1. Setup
tracker = DeepSort(max_age=40)
video_path = "/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4"
cap = cv2.VideoCapture(video_path)
output_dir = "output_frames"
os.makedirs(output_dir, exist_ok=True)
frame_num = 0  # Make sure to increment for each frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 2. Face detection
    faces = RetinaFace.detect_faces(frame)
    detections = []
    if isinstance(faces, dict):  # Only if faces were detected
        for key in faces:
            face = faces[key]
            x1, y1, x2, y2 = face['facial_area']
            conf = face.get('score', 0.99)  # Some models don't give score, so default to high
            detections.append(([x1, y1, x2-x1, y2-y1], conf, "face"))

    # 3. Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    # 4. Draw boxes and IDs
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

        # (Optional: add emotion detection and write above the box!)

    # 5. Save every processed frame
    cv2.imwrite(os.path.join(output_dir, f"frame_{frame_num:04d}.jpg"), frame)
    frame_num += 1

    # 6. Display for live feedback (optional)
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from retinaface import RetinaFace
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Configuration
CONF_THRESH = 0.8
MIN_FACE_SIZE = 20
RESIZE_WIDTH = 640
VIDEO_PATH = "/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4"
OUTPUT_DIR = "output_frames"
VIDEO_OUTPUT_PATH = "output_video.mp4"

# Initialize components
tracker = DeepSort(max_age=30, n_init=3)
cap = cv2.VideoCapture(VIDEO_PATH)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Get video properties for output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(VIDEO_OUTPUT_PATH, fourcc, fps, (frame_width, frame_height))

frame_num = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Store original frame for output
    output_frame = frame.copy()

    # Optimize detection with resizing
    orig_h, orig_w = frame.shape[:2]
    scale = RESIZE_WIDTH / orig_w
    new_h = int(orig_h * scale)
    frame_resized = cv2.resize(frame, (RESIZE_WIDTH, new_h))

    # Enhanced face detection
    faces = RetinaFace.detect_faces(frame_resized)
    detections = []
    
    if isinstance(faces, dict):
        for key, face in faces.items():
            x1, y1, x2, y2 = face['facial_area']
            conf = face.get('score', 0.99)
            w, h = x2 - x1, y2 - y1
            
            # Apply confidence and size filters
            if conf >= CONF_THRESH and w >= MIN_FACE_SIZE and h >= MIN_FACE_SIZE:
                detections.append(([x1, y1, w, h], conf, "face"))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame_resized)

    # Process tracks and draw on output frame
    for track in tracks:
        if not track.is_confirmed():
            continue
            
        track_id = track.track_id
        ltrb = track.to_ltrb()
        
        # Scale coordinates to original frame
        x1_orig = int(ltrb[0] / scale)
        y1_orig = int(ltrb[1] / scale)
        x2_orig = int(ltrb[2] / scale)
        y2_orig = int(ltrb[3] / scale)
        
        # Visualization
        cv2.rectangle(output_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0,255,0), 2)
        cv2.putText(output_frame, f"ID:{track_id}", (x1_orig, y1_orig-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

    # Save processed frame to folder
    cv2.imwrite(os.path.join(OUTPUT_DIR, f"frame_{frame_num:04d}.jpg"), output_frame)
    
    # Write frame to output video
    out_video.write(output_frame)
    
    # Display live feed
    cv2.imshow("Face Tracking", output_frame)
    frame_num += 1
    
    # Exit on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out_video.release()
cv2.destroyAllWindows()
print(f"Processing complete! Saved {frame_num} frames to '{OUTPUT_DIR}' and video to '{VIDEO_OUTPUT_PATH}'")


In [9]:
import cv2
import numpy as np
from retinaface import RetinaFace
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Configuration
CONF_THRESH = 0.4      # Lower threshold to catch more faces
MIN_FACE_SIZE = 20     # Minimum face width/height in pixels
RESIZE_WIDTH = 640     # Resize width for faster detection
OUTPUT_DIR = "output_frames"
VIDEO_OUTPUT_PATH = "output_video.mp4"
VIDEO_PATH = "/Users/bujjigaadu/Desktop/Data Analytics projects /Internship/resources/basketball.mp4"

# Initialize
tracker = DeepSort(max_age=30, n_init=3)
cap = cv2.VideoCapture(VIDEO_PATH)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Get video properties for output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(VIDEO_OUTPUT_PATH, fourcc, fps, (frame_width, frame_height))

frame_num = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Store original frame for output
    output_frame = frame.copy()

    # Preprocess: increase contrast and sharpen
    frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=20)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    frame = cv2.filter2D(frame, -1, kernel)

    # Resize for faster detection
    orig_h, orig_w = frame.shape[:2]
    scale = RESIZE_WIDTH / orig_w
    new_h = int(orig_h * scale)
    frame_resized = cv2.resize(frame, (RESIZE_WIDTH, new_h))

    # Detect faces with resnet50 backbone
    model = RetinaFace.build_model()  # or "mobilenet"
    faces = RetinaFace.detect_faces(frame_resized, model=model)
    detections = []

    if isinstance(faces, dict):
        for key, face in faces.items():
            conf = face.get('score', 0.99)
            x1, y1, x2, y2 = face['facial_area']
            w, h = x2 - x1, y2 - y1

            # Optional: filter by region of interest (ROI)
            # e.g., only faces in the court area
            # court_area = (0, 100, frame_resized.shape[1], frame_resized.shape[0]-50)
            # center_x, center_y = (x1 + x2)//2, (y1 + y2)//2
            # if (court_area[0] <= center_x <= court_area[2] and
            #     court_area[1] <= center_y <= court_area[3]):

            # Filter by confidence and size
            if conf > CONF_THRESH and w > MIN_FACE_SIZE and h > MIN_FACE_SIZE:
                detections.append(([x1, y1, w, h], conf, "face"))

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame_resized)

    # Draw tracks on original frame (scaled coordinates)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1_orig = int(ltrb[0] / scale)
        y1_orig = int(ltrb[1] / scale)
        x2_orig = int(ltrb[2] / scale)
        y2_orig = int(ltrb[3] / scale)
        cv2.rectangle(output_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), (0, 255, 0), 2)
        cv2.putText(output_frame, f"ID:{track_id}", (x1_orig, y1_orig-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Save processed frame and write to video
    cv2.imwrite(os.path.join(OUTPUT_DIR, f"frame_{frame_num:04d}.jpg"), output_frame)
    out_video.write(output_frame)
    frame_num += 1

    # Display live
    cv2.imshow("Face Tracking", output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out_video.release()
cv2.destroyAllWindows()
print(f"Processing complete! Saved {frame_num} frames to '{OUTPUT_DIR}' and video to '{VIDEO_OUTPUT_PATH}'")


Processing complete! Saved 322 frames to 'output_frames' and video to 'output_video.mp4'
